In [1]:
!git clone https://github.com/Ahnd6474/ESMS-VAE

Cloning into 'ESMS-VAE'...
remote: Enumerating objects: 626, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 626 (delta 51), reused 18 (delta 18), pack-reused 522 (from 2)
Receiving objects: 100% (626/626), 398.09 MiB | 22.66 MiB/s, done.
Resolving deltas: 100% (188/188), done.
Updating files: 100% (257/257), done.


In [2]:
%cd /kaggle/working/ESMS-VAE

/kaggle/working/ESMS-VAE


In [3]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 944.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 83.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing insta

In [4]:
import pandas as pd
vae_df=pd.read_csv('/kaggle/input/test-result/batch_results_with_rho.csv')
prev_df=pd.read_csv('/kaggle/input/dms-scores/merged_scores_substitutions_DMS.csv')
prev_df

,DMS_id,model_name,fold_variable_name,Spearman,MSE
0,A0A140D2T1_ZIKV_Sourisseau_2019,Embeddings - Augmented - ESM1v,fold_contiguous_5,0.012776,1.311910
1,A0A140D2T1_ZIKV_Sourisseau_2019,Embeddings - Augmented - ESM1v,fold_modulo_5,0.163152,1.001697
2,A0A140D2T1_ZIKV_Sourisseau_2019,Embeddings - Augmented - ESM1v,fold_random_5,0.232439,0.933645
3,A0A140D2T1_ZIKV_Sourisseau_2019,Embeddings - Augmented - MSA Transformer,fold_contiguous_5,0.429282,1.061674
4,A0A140D2T1_ZIKV_Sourisseau_2019,Embeddings - Augmented - MSA Transformer,fold_modulo_5,0.439994,0.937845
...,...,...,...,...,...
7156,YNZC_BACSU_Tsuboyama_2023_2JVD,OHE - Not augmented,fold_modulo_5,-0.070185,1.075730
7157,YNZC_BACSU_Tsuboyama_2023_2JVD,OHE - Not augmented,fold_random_5,0.690751,0.594683
7158,YNZC_BACSU_Tsuboyama_2023_2JVD,ProteinNPT,fold_contiguous_5,0.684150,0.472891
7159,YNZC_BACSU_Tsuboyama_2023_2JVD,ProteinNPT,fold_modulo_5,0.681212,0.474511


In [5]:
import pandas as pd
import matplotlib.pyplot as plt

# 1. Load the data (adjust paths if needed)
vae_df=pd.read_csv('/kaggle/input/test-result/batch_results_with_rho.csv')
prev_df=pd.read_csv('/kaggle/input/dms-scores/merged_scores_substitutions_DMS.csv')

# 2. Normalize IDs
vae_df['DMS_id'] = vae_df['dataset'].str[:-4]
vae_df['y']      = vae_df['rho']

# 3. Filter prev_df to only those IDs
common_ids = set(vae_df['DMS_id'])
prev_df     = prev_df[~prev_df['DMS_id'].isin(common_ids)]

to_do=prev_df['DMS_id'].to_list()
to_do=list(set(to_do))
print(len(to_do))

55


In [6]:
csv_path=list(map(lambda x:'/kaggle/input/dms-subs/DMS_ProteinGym_substitutions/'+x+'.csv',to_do))
csv_path

['/kaggle/input/dms-subs/DMS_ProteinGym_substitutions/PA_I34A1_Wu_2015.csv',
 '/kaggle/input/dms-subs/DMS_ProteinGym_substitutions/S22A1_HUMAN_Yee_2023_activity.csv',
 '/kaggle/input/dms-subs/DMS_ProteinGym_substitutions/SRC_HUMAN_Ahler_2019.csv',
 '/kaggle/input/dms-subs/DMS_ProteinGym_substitutions/BRCA2_HUMAN_Erwood_2022_HEK293T.csv',
 '/kaggle/input/dms-subs/DMS_ProteinGym_substitutions/POLG_HCVJF_Qi_2014.csv',
 '/kaggle/input/dms-subs/DMS_ProteinGym_substitutions/BRCA1_HUMAN_Findlay_2018.csv',
 '/kaggle/input/dms-subs/DMS_ProteinGym_substitutions/SC6A4_HUMAN_Young_2021.csv',
 '/kaggle/input/dms-subs/DMS_ProteinGym_substitutions/A4D664_9INFA_Soh_2019.csv',
 '/kaggle/input/dms-subs/DMS_ProteinGym_substitutions/SPIKE_SARS2_Starr_2020_expression.csv',
 '/kaggle/input/dms-subs/DMS_ProteinGym_substitutions/CAR11_HUMAN_Meitlis_2020_lof.csv',
 '/kaggle/input/dms-subs/DMS_ProteinGym_substitutions/Q837P4_ENTFA_Meier_2023.csv',
 '/kaggle/input/dms-subs/DMS_ProteinGym_substitutions/S22A1_HUMA

In [7]:
vae_df=pd.read_csv('/kaggle/input/test-result/batch_results_with_rho.csv')
vae_df['DMS_id']=vae_df['dataset'].str[:-4]
vae_df['Spearman']=vae_df['rho']
vae_df=vae_df[['DMS_id','Spearman','MSE']]

In [8]:
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, random_split
from vae_module import Config, Tokenizer, load_vae, encode_long_batch
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import spearmanr
rho=[]
lmse=[]
r_sq=[]
# --- 4. 모델 정의 ---
class MLPRegressor(nn.Module):
    def __init__(self, in_dim: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 1),
        )
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)
device = "cuda" if torch.cuda.is_available() else "cpu"
cfg = Config(model_path="models/vae_epoch380.pt", device=device)
tokenizer = Tokenizer.from_esm()
model = load_vae(
        cfg,
        vocab_size=len(tokenizer.vocab),
        pad_idx=tokenizer.pad_idx,
        bos_idx=tokenizer.bos_idx,
    )
print('loaded vae')
if device == "cuda" and torch.cuda.device_count() > 1:
        model = torch.nn.DataParallel(model)

for i in  range(len(csv_path)):
    df=pd.read_csv(csv_path[i])
    df['sequence']=df['mutated_sequence']
    df=df[['sequence','DMS_score']]
    

    """Generate long sequences, encode them, and train a small MLP."""
    
    
    sequences = df['sequence'].to_list()
    latent_stacks = encode_long_batch(model, sequences, tokenizer, cfg.max_len, overlap=256)
    Z = torch.stack([stack.mean(dim=0) for stack in latent_stacks]).cpu().numpy()
    
    dms = df['DMS_score'].to_list()
    y = np.array(dms, dtype=np.float32).reshape(-1,1)  # (n_samples, 1)
    print('Sequence encoding complete')
    sequences = df['sequence'].to_list()
    latent_stacks = encode_long_batch(model, sequences, tokenizer, cfg.max_len, overlap=256)
    Z = torch.stack([stack.mean(dim=0) for stack in latent_stacks]).cpu().numpy()
    
    dms = df['DMS_score'].to_list()
    y = np.array(dms, dtype=np.float32).reshape(-1,1)  # (n_samples, 1)
    print('Sequence encoding complete')
    X=Z
    # --- 2. 스케일링 ---
    X_scaler = StandardScaler()
    y_scaler = StandardScaler()
    
    # train/val split 전에 전체 데이터 스케일러에 fit – 
    # (데이터 누수를 우려하면 train만 fit 후 transform, val만 transform 해도 됩니다)
    X_scaled = X_scaler.fit_transform(X)
    y_scaled = y_scaler.fit_transform(y)
    
    # --- 3. Tensor 변환 및 Dataset 분할 ---
    X_tensor = torch.from_numpy(X_scaled).float()
    y_tensor = torch.from_numpy(y_scaled).float()
    
    dataset = TensorDataset(X_tensor, y_tensor)
    train_size = int(len(dataset) * 0.8)
    val_size   = len(dataset) - train_size
    train_ds, val_ds = random_split(dataset, [train_size, val_size])
    
    batch_size = 32
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_ds,   batch_size=batch_size)
    reg = MLPRegressor(in_dim=X.shape[1]).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(reg.parameters(), lr=1e-3)
    
    
    num_epochs = 300
    best_val_loss = float('inf')
    best_epoch = 0
    save_path = f"/kaggle/working/{to_do[i]}_best_model.pth"
    
    for epoch in range(1, num_epochs + 1):
        # --- train ---
        reg.train()
        train_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = reg(xb)
            loss = criterion(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * xb.size(0)
        train_loss /= len(train_loader.dataset)
    
        # --- validation ---
        reg.eval()
        val_loss = 0.0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                pred = reg(xb)
                val_loss += criterion(pred, yb).item() * xb.size(0)
        val_loss /= len(val_loader.dataset)
    
    
    print(f"\nBest model at epoch {best_epoch} with Val Loss: {best_val_loss:.4f}")
    
    

    # 2) 평가 모드
    reg.eval()
    y_true_scaled, y_pred_scaled = [], []

    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(device)
            preds = reg(xb).cpu().numpy().flatten()
            y_pred_scaled.extend(preds)
            y_true_scaled.extend(yb.cpu().numpy().flatten())
    
    y_true_scaled = np.array(y_true_scaled).reshape(-1, 1)
    y_pred_scaled = np.array(y_pred_scaled).reshape(-1, 1)
    
    # 3) 스케일 역변환
    y_true = y_scaler.inverse_transform(y_true_scaled).flatten()
    y_pred = y_scaler.inverse_transform(y_pred_scaled).flatten()
    
    # 4) 지표 계산
    mse   = mean_squared_error(y_true, y_pred)
    r2    = r2_score(y_true, y_pred)
    spearman_corr, _ = spearmanr(y_true, y_pred)
    print(f"  MSE:               {mse:.4f}")
    print(f"  R²:                {r2:.4f}")
    print(f"  Spearman Corr:     {spearman_corr:.4f}")
    rho.append(spearman_corr)
    lmse.append(mse)
    r_sq.append(r2)
    
df1=pd.DataFrame({'DMS_id':to_do,'Spearman':rho,'MSE':lmse})


df = pd.concat([df1, vae_df], ignore_index=True)
df.to_csv('/kaggle/working/dms_score.csv')
print(df['Spearman'].mean())

2025-06-27 05:37:16 vae_module.loader [INFO] Loaded VAE from models/vae_epoch380.pt on cuda


loaded vae


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:508: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. We recommend specifying layout=torch.jagged when constructing a nested tensor, as this layout receives active development, has better operator coverage, and works with torch.compile. (Triggered internally at /pytorch/aten/src/ATen/NestedTensorImpl.cpp:178.)
  output = torch._nested_tensor_from_mask(
/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5962: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5962: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Sequence encoding complete
Sequence encoding complete

Best model at epoch 0 with Val Loss: inf
  MSE:               0.2011
  R²:                -0.3304
  Spearman Corr:     0.2810


/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5962: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


...

Sequence encoding complete
Sequence encoding complete

Best model at epoch 0 with Val Loss: inf
  MSE:               0.0615
  R²:                0.6045
  Spearman Corr:     0.7857


/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5962: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Sequence encoding complete
Sequence encoding complete

Best model at epoch 0 with Val Loss: inf
  MSE:               0.9364
  R²:                -0.1245
  Spearman Corr:     0.2520


/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5962: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Sequence encoding complete
Sequence encoding complete

Best model at epoch 0 with Val Loss: inf
  MSE:               0.0519
  R²:                0.2120
  Spearman Corr:     0.3279
0.6885984690686742
